In [1]:
import sys
sys.path.append("../classes")

In [2]:
import pickle
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

import numpy as np, pandas as pd
from geno_classifier import *
from geno_utils import parse_database
from itertools import starmap

import GEOparse

In [4]:
diabetes_dataset = GEOparse.get_GEO(filepath='../datasets/GDS3782_full.soft').table

01-Feb-2018 12:55:21 INFO GEOparse - Parsing ../datasets/GDS3782_full.soft: 
01-Feb-2018 12:55:21 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 12:55:21 DEBUG GEOparse - DATASET: GDS3782
01-Feb-2018 12:55:21 DEBUG GEOparse - SUBSET: GDS3782_1
01-Feb-2018 12:55:21 DEBUG GEOparse - SUBSET: GDS3782_2
01-Feb-2018 12:55:21 DEBUG GEOparse - ANNOTATION: 
01-Feb-2018 12:55:21 ERROR GEOparse - Cannot recognize type Annotation
01-Feb-2018 12:55:21 DEBUG GEOparse - DATASET: GDS3782
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:88 DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.


In [5]:
healthy_raw = """
#GSM524151 = Value for GSM524151: beta-cells_non-diabetic condition_donor1; src: human beta-cells_non-diabetic
#GSM524152 = Value for GSM524152: beta-cells_non-diabetic condition_donor2; src: human beta-cells_non-diabetic
#GSM524153 = Value for GSM524153: beta-cells_non-diabetic condition_donor3; src: human beta-cells_non-diabetic
#GSM524154 = Value for GSM524154: beta-cells_non-diabetic condition_donor4; src: human beta-cells_non-diabetic
#GSM524155 = Value for GSM524155: beta-cells_non-diabetic condition_donor5; src: human beta-cells_non-diabetic
#GSM524156 = Value for GSM524156: beta-cells_non-diabetic condition_donor6; src: human beta-cells_non-diabetic
#GSM524157 = Value for GSM524157: beta-cells_non-diabetic condition_donor7; src: human beta-cells_non-diabetic
#GSM524158 = Value for GSM524158: beta-cells_non-diabetic condition_donor8; src: human beta-cells_non-diabetic
#GSM524159 = Value for GSM524159: beta-cells_non-diabetic condition_donor9; src: human beta-cells_non-diabetic
#GSM524160 = Value for GSM524160: beta-cells_non-diabetic condition_donor10; src: human beta-cells_non-diabetic"""

In [6]:
unhealthy_raw = """
#GSM524161 = Value for GSM524161: beta-cells_diabetic condition_donor1; src: human beta-cells_diabetic
#GSM524162 = Value for GSM524162: beta-cells_diabetic condition_donor2; src: human beta-cells_diabetic
#GSM524163 = Value for GSM524163: beta-cells_diabetic condition_donor3; src: human beta-cells_diabetic
#GSM524164 = Value for GSM524164: beta-cells_diabetic condition_donor4; src: human beta-cells_diabetic
#GSM524165 = Value for GSM524165: beta-cells_diabetic condition_donor5; src: human beta-cells_diabetic
#GSM524166 = Value for GSM524166: beta-cells_diabetic condition_donor6; src: human beta-cells_diabetic
#GSM524167 = Value for GSM524167: beta-cells_diabetic condition_donor7; src: human beta-cells_diabetic
#GSM524168 = Value for GSM524168: beta-cells_diabetic condition_donor8; src: human beta-cells_diabetic
#GSM524169 = Value for GSM524169: beta-cells_diabetic condition_donor9; src: human beta-cells_diabetic
#GSM524170 = Value for GSM524170: beta-cells_diabetic condition_donor10; src: human beta-cells_diabetic
"""

In [7]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

In [8]:
len(healthy),len(unhealthy)

(10, 10)

In [9]:
# healthy = list(healthy)[:16]
# unhealthy = list(unhealthy)[:16]

# # pickle.dump({"helthy":healthy,
# #              "unhealthy":unhealthy},
# #            open("colon_cancer_subset","wb"))

In [10]:
healthy = set(healthy)
unhealthy = set(unhealthy)

In [11]:
sample_columns = list(healthy | unhealthy)

In [12]:
cle = diabetes_dataset[["ID_REF","IDENTIFIER"]+sample_columns]

In [13]:
cle.dropna(inplace=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
cle[cle.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM524155,GSM524167,GSM524154
0,1053_3p_at,RFC2,12.06,10.50,3.37
1,117_3p_at,HSPA6,45.94,42.72,44.11
2,1494_3p_f_at,CYP2A6,63.53,42.79,61.92
3,1552275_3p_s_at,PXK,24.16,12.10,24.62
4,1552281_3p_at,SLC39A5,28.19,43.59,35.96


In [15]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [ ]:
X, y = parse_database(diabetes_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)

In [ ]:
pickle.dump({"X":X,"y":y}, open("diabetes_Xy", "wb"))

## HBV

In [16]:
HBV_dataset = GEOparse.get_GEO(filepath='../datasets/GDS4387_full.soft').table

01-Feb-2018 12:55:35 INFO GEOparse - Parsing ../datasets/GDS4387_full.soft: 
01-Feb-2018 12:55:35 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 12:55:35 DEBUG GEOparse - DATASET: GDS4387
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_1
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_2
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_3
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_4
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_5
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_6
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_7
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_8
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_9
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_10
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_11
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_12
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_13
01-Feb-2018 12:55:35 DEBUG GEOparse - SUBSET: GDS4387_14
01-Feb-2018 12:55:35 DEBUG 

In [17]:
healthy_raw = """#GSM952520 = Value for GSM952520: normal liver from patient 112; src: specimen obtained from explanted liver Patient 112
#GSM952519 = Value for GSM952519: normal liver from patient 101; src: specimen obtained from explanted liver Patient 101
#GSM952518 = Value for GSM952518: normal liver from patient 62; src: specimen obtained from explanted liver Patient 62
#GSM952517 = Value for GSM952517: normal liver from patient 60; src: specimen obtained from explanted liver Patient 60
#GSM952516 = Value for GSM952516: normal liver from patient 37; src: specimen obtained from explanted liver Patient 37
#GSM952515 = Value for GSM952515: normal liver from patient 29; src: specimen obtained from explanted liver Patient 29
#GSM952514 = Value for GSM952514: normal liver from patient 20; src: specimen obtained from explanted liver Patient 20
#GSM952513 = Value for GSM952513: normal liver from patient 18; src: specimen obtained from explanted liver Patient 18
#GSM952512 = Value for GSM952512: normal liver from patient 16; src: specimen obtained from explanted liver Patient 16
#GSM952511 = Value for GSM952511: normal liver from patient 8; src: specimen obtained from explanted liver Patient 8
"""

In [18]:
unhealthy_raw = """#GSM952534 = Value for GSM952534: HBV-associated acute liver failure liver from patient 241-1; src: specimen obtained from explanted liver Patient 241
#GSM952535 = Value for GSM952535: HBV-associated acute liver failure liver from patient 241-2; src: specimen obtained from explanted liver Patient 241
#GSM952536 = Value for GSM952536: HBV-associated acute liver failure liver from patient 241-3; src: specimen obtained from explanted liver Patient 241
#GSM952537 = Value for GSM952537: HBV-associated acute liver failure liver from patient 241-4; src: specimen obtained from explanted liver Patient 241
#GSM952529 = Value for GSM952529: HBV-associated acute liver failure liver from patient 219-1; src: specimen obtained from explanted liver Patient 219
#GSM952530 = Value for GSM952530: HBV-associated acute liver failure liver from patient 219-2; src: specimen obtained from explanted liver Patient 219
#GSM952531 = Value for GSM952531: HBV-associated acute liver failure liver from patient 219-3; src: specimen obtained from explanted liver Patient 219
#GSM952532 = Value for GSM952532: HBV-associated acute liver failure liver from patient 219-4; src: specimen obtained from explanted liver Patient 219
#GSM952533 = Value for GSM952533: HBV-associated acute liver failure liver from patient 219-5; src: specimen obtained from explanted liver Patient 219
#GSM952525 = Value for GSM952525: HBV-associated acute liver failure liver from patient 32-1; src: specimen obtained from explanted liver Patient 32
#GSM952526 = Value for GSM952526: HBV-associated acute liver failure liver from patient 32-2; src: specimen obtained from explanted liver Patient 32
#GSM952527 = Value for GSM952527: HBV-associated acute liver failure liver from patient 32-3; src: specimen obtained from explanted liver Patient 32
#GSM952528 = Value for GSM952528: HBV-associated acute liver failure liver from patient 32-4; src: specimen obtained from explanted liver Patient 32
#GSM952521 = Value for GSM952521: HBV-associated acute liver failure liver from patient 31-1; src: specimen obtained from explanted liver Patient 31
#GSM952522 = Value for GSM952522: HBV-associated acute liver failure liver from patient 31-2; src: specimen obtained from explanted liver Patient 31
#GSM952523 = Value for GSM952523: HBV-associated acute liver failure liver from patient 31-3; src: specimen obtained from explanted liver Patient 31
#GSM952524 = Value for GSM952524: HBV-associated acute liver failure liver from patient 31-4; src: specimen obtained from explanted liver Patient 31
"""

In [20]:
X, y = parse_database(HBV_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("HBV_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done  15 out of  27 | elapsed: 19.1min remaining: 15.3min
[Parallel(n_jobs=-1)]: Done  18 out of  27 | elapsed: 21.0min remaining: 10.5min
[Parallel(n_jobs=-1)]: Done  21 out of  27 | elapsed: 21.1min remaining:  6.0min
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed: 21.2min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 21.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 21.6min finished


GSM952534 added with length 19266
GSM952532 added with length 19266
GSM952533 added with length 19266
GSM952528 added with length 19266
GSM952524 added with length 19266
GSM952536 added with length 19266
GSM952522 added with length 19266
GSM952514 added with length 19266
GSM952530 added with length 19266
GSM952529 added with length 19266
GSM952537 added with length 19266
GSM952519 added with length 19266
GSM952526 added with length 19266
GSM952531 added with length 19266
GSM952517 added with length 19266
GSM952515 added with length 19266
GSM952520 added with length 19266
GSM952518 added with length 19266
GSM952513 added with length 19266
GSM952521 added with length 19266
GSM952511 added with length 19266
GSM952535 added with length 19266
GSM952525 added with length 19266
GSM952516 added with length 19266
GSM952523 added with length 19266
GSM952512 added with length 19266
GSM952527 added with length 19266


## Pancreatic Cancer

In [26]:
pancreatic_dataset = GEOparse.get_GEO(filepath='../datasets/GDS4100_full.soft').table

01-Feb-2018 13:20:47 INFO GEOparse - Parsing ../datasets/GDS4100_full.soft: 
01-Feb-2018 13:20:47 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 13:20:47 DEBUG GEOparse - DATASET: GDS4100
01-Feb-2018 13:20:47 DEBUG GEOparse - SUBSET: GDS4100_1
01-Feb-2018 13:20:47 DEBUG GEOparse - SUBSET: GDS4100_2
01-Feb-2018 13:20:47 DEBUG GEOparse - ANNOTATION: 
01-Feb-2018 13:20:47 ERROR GEOparse - Cannot recognize type Annotation
01-Feb-2018 13:20:47 DEBUG GEOparse - DATASET: GDS4100
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:88 DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.


In [27]:
healthy_raw = """
#GSM356808 = Value for GSM356808: Healthy control saliva-1; src: saliva
#GSM356809 = Value for GSM356809: Healthy control saliva-2; src: saliva
#GSM356810 = Value for GSM356810: Healthy control saliva-3; src: saliva
#GSM356811 = Value for GSM356811: Healthy control saliva-4; src: saliva
#GSM356812 = Value for GSM356812: Healthy control saliva-5; src: saliva
#GSM356813 = Value for GSM356813: Healthy control saliva-6; src: saliva
#GSM356814 = Value for GSM356814: Healthy control saliva-7; src: saliva
#GSM356815 = Value for GSM356815: Healthy control saliva-8; src: saliva
#GSM356816 = Value for GSM356816: Healthy control saliva-9; src: saliva
#GSM356817 = Value for GSM356817: Healthy control saliva-10; src: saliva
#GSM356818 = Value for GSM356818: Healthy control saliva-11; src: saliva
#GSM356819 = Value for GSM356819: Healthy control saliva-12; src: saliva
"""

In [28]:
unhealthy_raw = """
#GSM356796 = Value for GSM356796: pancreatic cancer saliva-1; src: saliva
#GSM356797 = Value for GSM356797: pancreatic cancer saliva-2; src: saliva
#GSM356798 = Value for GSM356798: pancreatic cancer saliva-3; src: saliva
#GSM356799 = Value for GSM356799: pancreatic cancer saliva-4; src: saliva
#GSM356800 = Value for GSM356800: pancreatic cancer saliva-5; src: saliva
#GSM356801 = Value for GSM356801: pancreatic cancer saliva-6; src: saliva
#GSM356802 = Value for GSM356802: pancreatic cancer saliva-7; src: saliva
#GSM356803 = Value for GSM356803: pancreatic cancer saliva-8; src: saliva
#GSM356804 = Value for GSM356804: pancreatic cancer saliva-9; src: saliva
#GSM356805 = Value for GSM356805: pancreatic cancer saliva-10; src: saliva
#GSM356806 = Value for GSM356806: pancreatic cancer saliva-11; src: saliva
#GSM356807 = Value for GSM356807: pancreatic cancer saliva-12; src: saliva
"""

In [29]:
X, y = parse_database(pancreatic_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("pancreatic_cancer_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  12 out of  24 | elapsed:  4.5min remaining:  4.5min
[Parallel(n_jobs=-1)]: Done  15 out of  24 | elapsed:  4.6min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed:  4.6min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed:  4.6min remaining:   39.6s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  4.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  4.6min finished


GSM356818 added with length 5777
GSM356815 added with length 5777
GSM356810 added with length 5777
GSM356799 added with length 5777
GSM356802 added with length 5777
GSM356809 added with length 5777
GSM356811 added with length 5777
GSM356806 added with length 5777
GSM356801 added with length 5777
GSM356800 added with length 5777
GSM356807 added with length 5777
GSM356804 added with length 5777
GSM356812 added with length 5777
GSM356803 added with length 5777
GSM356813 added with length 5777
GSM356808 added with length 5777
GSM356797 added with length 5777
GSM356817 added with length 5777
GSM356805 added with length 5777
GSM356796 added with length 5777
GSM356816 added with length 5777
GSM356814 added with length 5777
GSM356819 added with length 5777
GSM356798 added with length 5777


In [30]:
renal_carcinoma_dataset = GEOparse.get_GEO(filepath='../datasets/GDS2881_full.soft').table

01-Feb-2018 13:26:01 INFO GEOparse - Parsing ../datasets/GDS2881_full.soft: 
01-Feb-2018 13:26:01 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 13:26:01 DEBUG GEOparse - DATASET: GDS2881
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_1
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_2
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_3
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_4
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_5
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_6
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_7
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_8
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_9
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_10
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_11
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_12
01-Feb-2018 13:26:01 DEBUG GEOparse - SUBSET: GDS2881_13
01-Feb-2018 13:26:01 DEBUG GEOparse - ANNOTATION: 
01-Feb-2018 13:26:01 ERROR GEOpar

In [31]:
healthy_raw = """
#GSM146798 = Value for GSM146798: Stage 1, PT#2, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146800 = Value for GSM146800: Stage 1, PT#3, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146802 = Value for GSM146802: Stage 1, PT#4, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146804 = Value for GSM146804: Stage 1, PT#5,Normal (HG-U133B); src: Human Renal Epithelium
#GSM146806 = Value for GSM146806: Stage 1, PT#6, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146809 = Value for GSM146809: Stage 2, PT#8, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146810 = Value for GSM146810: Stage 2, PT#9, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146812 = Value for GSM146812: Stage 2, PT#10, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146814 = Value for GSM146814: Stage 2, PT#11, Normal (HG-U133B); src: Human Renal Epithelium
#GSM146816 = Value for GSM146816: Stage 2, PT#12, Normal (HG-U133B); src: Human Renal Epithelium
"""

In [32]:
unhealthy_raw = """
#GSM146799 = Value for GSM146799: Stage 1, PT#2, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146801 = Value for GSM146801: Stage 1, PT#3, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146803 = Value for GSM146803: Stage 1, PT#4,Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146805 = Value for GSM146805: Stage 1, PT#5, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146807 = Value for GSM146807: Stage 1, PT#6,Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146808 = Value for GSM146808: Stage 2, PT#8, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146811 = Value for GSM146811: Stage 2, PT#9, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146813 = Value for GSM146813: Stage 2, PT#10, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146815 = Value for GSM146815: Stage 2, PT#11, Tumor (HG-U133B); src: Human Renal Epithelium
#GSM146817 = Value for GSM146817: Stage 2, PT#12, Tumor (HG-U133B); src: Human Renal Epithelium"""

In [33]:
X, y = parse_database(renal_carcinoma_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("renal_carcinoma_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  9.3min remaining: 14.0min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  9.6min remaining:  7.9min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:  9.7min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  9.8min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  9.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  9.9min finished


GSM146816 added with length 9488
GSM146799 added with length 9488
GSM146812 added with length 9488
GSM146803 added with length 9488
GSM146810 added with length 9488
GSM146809 added with length 9488
GSM146817 added with length 9488
GSM146805 added with length 9488
GSM146815 added with length 9488
GSM146804 added with length 9488
GSM146811 added with length 9488
GSM146807 added with length 9488
GSM146802 added with length 9488
GSM146800 added with length 9488
GSM146808 added with length 9488
GSM146806 added with length 9488
GSM146814 added with length 9488
GSM146798 added with length 9488
GSM146813 added with length 9488
GSM146801 added with length 9488


## Glioma platlets

In [36]:
glioma_dataset = GEOparse.get_GEO(filepath='../datasets/GDS5181_full.soft').table

01-Feb-2018 13:38:41 INFO GEOparse - Parsing ../datasets/GDS5181_full.soft: 
01-Feb-2018 13:38:41 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 13:38:41 DEBUG GEOparse - DATASET: GDS5181
01-Feb-2018 13:38:41 DEBUG GEOparse - SUBSET: GDS5181_1
01-Feb-2018 13:38:41 DEBUG GEOparse - SUBSET: GDS5181_2
01-Feb-2018 13:38:41 DEBUG GEOparse - ANNOTATION: 
01-Feb-2018 13:38:41 ERROR GEOparse - Cannot recognize type Annotation
01-Feb-2018 13:38:41 DEBUG GEOparse - DATASET: GDS5181
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [34]:
healthy_raw = """
#GSM769920 = Value for GSM769920: THB_Control_s1; src: THB_Control_s1
#GSM769921 = Value for GSM769921: THB_Control_s2; src: THB_Control_s2
#GSM769922 = Value for GSM769922: THB_Control_s3; src: THB_Control_s3
#GSM769923 = Value for GSM769923: THB_Control_s4; src: THB_Control_s4
#GSM769924 = Value for GSM769924: THB_Control_s5; src: THB_Control_s5
#GSM769925 = Value for GSM769925: THB_Control_s6; src: THB_Control_s6
#GSM769926 = Value for GSM769926: THB_Control_s7; src: THB_Control_s7
#GSM769927 = Value for GSM769927: THB_Control_s8; src: THB_Control_s8
#GSM769928 = Value for GSM769928: THB_Control_s9; src: THB_Control_s9
#GSM769929 = Value for GSM769929: THB_Control_s10; src: THB_Control_s10
#GSM769930 = Value for GSM769930: THB_Control_s11; src: THB_Control_s11
#GSM769931 = Value for GSM769931: THB_Control_s12; src: THB_Control_s12
"""

In [35]:
unhealthy_raw ="""
#GSM769932 = Value for GSM769932: THB_Glioma_s1; src: THB_Glioma_s1
#GSM769933 = Value for GSM769933: THB_Glioma_s2; src: THB_Glioma_s2
#GSM769934 = Value for GSM769934: THB_Glioma_s3; src: THB_Glioma_s3
#GSM769935 = Value for GSM769935: THB_Glioma_s4; src: THB_Glioma_s4
#GSM769936 = Value for GSM769936: THB_Glioma_s5; src: THB_Glioma_s5
#GSM769937 = Value for GSM769937: THB_Glioma_s6; src: THB_Glioma_s6
#GSM769938 = Value for GSM769938: THB_Glioma_s7; src: THB_Glioma_s7
#GSM769939 = Value for GSM769939: THB_Glioma_s8; src: THB_Glioma_s8"""

In [37]:
X, y = parse_database(glioma_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("glioma_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed: 17.5min remaining: 26.3min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed: 18.8min remaining: 15.4min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed: 19.0min remaining:  8.1min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 19.8min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.8min finished


GSM769935 added with length 18126
GSM769931 added with length 18126
GSM769934 added with length 18126
GSM769936 added with length 18126
GSM769925 added with length 18126
GSM769933 added with length 18126
GSM769932 added with length 18126
GSM769927 added with length 18126
GSM769921 added with length 18126
GSM769939 added with length 18126
GSM769926 added with length 18126
GSM769930 added with length 18126
GSM769937 added with length 18126
GSM769922 added with length 18126
GSM769938 added with length 18126
GSM769924 added with length 18126
GSM769923 added with length 18126
GSM769928 added with length 18126
GSM769929 added with length 18126
GSM769920 added with length 18126


In [38]:
esophageal_carcinoma_dataset = GEOparse.get_GEO(filepath='../datasets/GDS3838_full.soft').table

01-Feb-2018 13:58:46 INFO GEOparse - Parsing ../datasets/GDS3838_full.soft: 
01-Feb-2018 13:58:46 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 13:58:46 DEBUG GEOparse - DATASET: GDS3838
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_1
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_2
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_3
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_4
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_5
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_6
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_7
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_8
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_9
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_10
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_11
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_12
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_13
01-Feb-2018 13:58:46 DEBUG GEOparse - SUBSET: GDS3838_14
01-Feb-2018 13:58:46 DEBUG 

In [39]:
healthy_raw = """
#GSM509787 = Value for GSM509787: normal adjacent esophageal tissue from patient E1507; src: patient E1507
#GSM509788 = Value for GSM509788: normal adjacent esophageal tissue from patient E1520; src: patient E1520
#GSM509789 = Value for GSM509789: normal adjacent esophageal tissue from patient E1521; src: patient E1521
#GSM509790 = Value for GSM509790: normal adjacent esophageal tissue from patient E1532; src: patient E1532
#GSM509791 = Value for GSM509791: normal adjacent esophageal tissue from patient E1535; src: patient E1535
#GSM509792 = Value for GSM509792: normal adjacent esophageal tissue from patient E1542; src: patient E1542
#GSM509793 = Value for GSM509793: normal adjacent esophageal tissue from patient E1546; src: patient E1546
#GSM509794 = Value for GSM509794: normal adjacent esophageal tissue from patient E1566; src: patient E1566
#GSM509795 = Value for GSM509795: normal adjacent esophageal tissue from patient E1584; src: patient E1584
#GSM509796 = Value for GSM509796: normal adjacent esophageal tissue from patient E1589; src: patient E1589
"""

In [42]:
unhealthy_raw ="""
#GSM509804 = Value for GSM509804: esophageal squamous cell carcinoma from patient E1507; src: patient E1507
#GSM509805 = Value for GSM509805: esophageal squamous cell carcinoma from patient E1520; src: patient E1520
#GSM509806 = Value for GSM509806: esophageal squamous cell carcinoma from patient E1521; src: patient E1521
#GSM509807 = Value for GSM509807: esophageal squamous cell carcinoma from patient E1532; src: patient E1532
#GSM509808 = Value for GSM509808: esophageal squamous cell carcinoma from patient E1535; src: patient E1535
#GSM509809 = Value for GSM509809: esophageal squamous cell carcinoma from patient E1542; src: patient E1542
#GSM509810 = Value for GSM509810: esophageal squamous cell carcinoma from patient E1546; src: patient E1546
#GSM509811 = Value for GSM509811: esophageal squamous cell carcinoma from patient E1566; src: patient E1566
#GSM509812 = Value for GSM509812: esophageal squamous cell carcinoma from patient E1584; src: patient E1584
#GSM509813 = Value for GSM509813: esophageal squamous cell carcinoma from patient E1589; src: patient E1589
#GSM509814 = Value for GSM509814: esophageal squamous cell carcinoma from patient E1603; src: patient E1603
#GSM509815 = Value for GSM509815: esophageal squamous cell carcinoma from patient E1610; src: patient E1610
#GSM509816 = Value for GSM509816: esophageal squamous cell carcinoma from patient E1614; src: patient E1614
#GSM509817 = Value for GSM509817: esophageal squamous cell carcinoma from patient E1635; src: patient E1635
#GSM509818 = Value for GSM509818: esophageal squamous cell carcinoma from patient E1709; src: patient E1709
#GSM509819 = Value for GSM509819: esophageal squamous cell carcinoma from patient E1796; src: patient E1796
#GSM509820 = Value for GSM509820: esophageal squamous cell carcinoma from patient E2644; src: patient E2644"""

In [43]:
X, y = parse_database(esophageal_carcinoma_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("esophageal_carcinoma_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  15 out of  27 | elapsed:  7.5min remaining:  6.0min
[Parallel(n_jobs=-1)]: Done  18 out of  27 | elapsed:  7.9min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done  21 out of  27 | elapsed:  7.9min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:  7.9min remaining:   59.2s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  8.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  8.0min finished


GSM509794 added with length 12666
GSM509811 added with length 12666
GSM509818 added with length 12666
GSM509810 added with length 12666
GSM509805 added with length 12666
GSM509806 added with length 12666
GSM509819 added with length 12666
GSM509789 added with length 12666
GSM509814 added with length 12666
GSM509812 added with length 12666
GSM509820 added with length 12666
GSM509813 added with length 12666
GSM509790 added with length 12666
GSM509816 added with length 12666
GSM509795 added with length 12666
GSM509804 added with length 12666
GSM509788 added with length 12666
GSM509809 added with length 12666
GSM509817 added with length 12666
GSM509796 added with length 12666
GSM509791 added with length 12666
GSM509808 added with length 12666
GSM509792 added with length 12666
GSM509815 added with length 12666
GSM509787 added with length 12666
GSM509793 added with length 12666
GSM509807 added with length 12666


## H1N1

In [44]:
H1N1_dataset = GEOparse.get_GEO(filepath='../datasets/GDS4240_full.soft').table

01-Feb-2018 14:07:14 INFO GEOparse - Parsing ../datasets/GDS4240_full.soft: 
01-Feb-2018 14:07:14 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 14:07:14 DEBUG GEOparse - DATASET: GDS4240
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_1
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_2
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_3
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_4
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_5
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_6
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_7
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_8
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_9
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_10
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_11
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_12
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_13
01-Feb-2018 14:07:14 DEBUG GEOparse - SUBSET: GDS4240_14
01-Feb-2018 14:07:14 DEBUG 

In [45]:
healthy_raw = """
#GSM670463 = Value for GSM670463: Blood_Control 1; src: Blood
#GSM670464 = Value for GSM670464: Blood_Control 2; src: Blood
#GSM670465 = Value for GSM670465: Blood_Control 3; src: Blood
#GSM670466 = Value for GSM670466: Blood_Control 4; src: Blood
#GSM670467 = Value for GSM670467: Blood_Control 5; src: Blood
#GSM670468 = Value for GSM670468: Blood_Control 6; src: Blood
#GSM670469 = Value for GSM670469: Blood_Control 7; src: Blood
"""

In [46]:
unhealthy_raw = """
#GSM670449 = Value for GSM670449: blood_day0_patient1; src: Blood
#GSM670450 = Value for GSM670450: blood_day6_patient1; src: Blood
#GSM670451 = Value for GSM670451: blood_day0_patient2; src: Blood
#GSM670452 = Value for GSM670452: blood_day6_patient2; src: Blood
#GSM670453 = Value for GSM670453: blood_day0_patient3; src: Blood
#GSM670454 = Value for GSM670454: blood_day6_patient3; src: Blood
#GSM670455 = Value for GSM670455: blood_day0_patient4; src: Blood
#GSM670456 = Value for GSM670456: blood_day6_patient4; src: Blood
#GSM670457 = Value for GSM670457: blood_day0_patient5; src: Blood
#GSM670458 = Value for GSM670458: blood_day6_patient5; src: Blood
#GSM670459 = Value for GSM670459: blood_day0_patient6; src: Blood
#GSM670460 = Value for GSM670460: blood_day6_patient6; src: Blood
#GSM670461 = Value for GSM670461: blood_day0_patient7; src: Blood
#GSM670462 = Value for GSM670462: blood_day6_patient7; src: Blood
"""

In [48]:
X, y = parse_database(H1N1_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("H1N1_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done   9 out of  21 | elapsed: 14.4min remaining: 19.2min
[Parallel(n_jobs=-1)]: Done  12 out of  21 | elapsed: 14.5min remaining: 10.9min
[Parallel(n_jobs=-1)]: Done  15 out of  21 | elapsed: 14.6min remaining:  5.8min
[Parallel(n_jobs=-1)]: Done  18 out of  21 | elapsed: 14.9min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed: 15.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed: 15.0min finished


GSM670466 added with length 19182
GSM670468 added with length 19182
GSM670450 added with length 19182
GSM670455 added with length 19182
GSM670463 added with length 19182
GSM670457 added with length 19182
GSM670465 added with length 19182
GSM670458 added with length 19182
GSM670449 added with length 19182
GSM670464 added with length 19182
GSM670460 added with length 19182
GSM670461 added with length 19182
GSM670454 added with length 19182
GSM670462 added with length 19182
GSM670456 added with length 19182
GSM670452 added with length 19182
GSM670451 added with length 19182
GSM670467 added with length 19182
GSM670459 added with length 19182
GSM670453 added with length 19182
GSM670469 added with length 19182


## Psoriasis

In [7]:
psoriasis_dataset = GEOparse.get_GEO(filepath='../datasets/GDS4602_full.soft').table

05-Feb-2018 23:05:41 INFO GEOparse - Parsing ../datasets/GDS4602_full.soft: 
05-Feb-2018 23:05:41 DEBUG GEOparse - DATABASE: Geo
05-Feb-2018 23:05:41 DEBUG GEOparse - DATASET: GDS4602
05-Feb-2018 23:05:41 DEBUG GEOparse - SUBSET: GDS4602_1
05-Feb-2018 23:05:41 DEBUG GEOparse - SUBSET: GDS4602_2
05-Feb-2018 23:05:41 DEBUG GEOparse - SUBSET: GDS4602_3
05-Feb-2018 23:05:41 DEBUG GEOparse - SUBSET: GDS4602_4
05-Feb-2018 23:05:41 DEBUG GEOparse - SUBSET: GDS4602_5
05-Feb-2018 23:05:41 DEBUG GEOparse - ANNOTATION: 
05-Feb-2018 23:05:41 ERROR GEOparse - Cannot recognize type Annotation
05-Feb-2018 23:05:41 DEBUG GEOparse - DATASET: GDS4602
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:88 DtypeWarning: Column

In [27]:
unhealthy_raw = "GSM337261,GSM337262,GSM337263,GSM337264,GSM337265,GSM337266,GSM337267,GSM337268,GSM337269,GSM337270,GSM337271,GSM337272,GSM337273,GSM337274,GSM337275,GSM337276,GSM337277,GSM337278,GSM337279,GSM337280,GSM337281,GSM337282,GSM337283,GSM337284,GSM337285,GSM337286,GSM337287,GSM337288,GSM337289,GSM337290,GSM337291,GSM337292,GSM337293,GSM337294,GSM337295,GSM337296,GSM337297,GSM337298,GSM337299,GSM337300,GSM337301,GSM337302,GSM337303,GSM337304,GSM337305,GSM337306,GSM337307,GSM337308,GSM337309,GSM337310,GSM337311,GSM337312,GSM337313,GSM337314,GSM337315,GSM337316,GSM337317,GSM337318,GSM337319,GSM337320,GSM337321,GSM337322,GSM337323,GSM337324,GSM337325,GSM337326,GSM337327,GSM337328,GSM337329,GSM337330,GSM337331,GSM337332,GSM337333,GSM337334,GSM337335,GSM337336,GSM337337,GSM337338,GSM337339,GSM337340,GSM337341,GSM337342,GSM337343,GSM337344,GSM337345,GSM337346,GSM337347,GSM337348,GSM337349,GSM337350,GSM337351,GSM337352,GSM337353,GSM337354,GSM337355,GSM337356,GSM337357,GSM337358,GSM337359,GSM337360,GSM337361,GSM337362,GSM337363,GSM337364,GSM337365,GSM337366,GSM337367,GSM337368,GSM337369,GSM337370,GSM337371,GSM337372,GSM337373,GSM337374,GSM337375,GSM337376"\
                        .split(",")
unhealthy_raw = "\n".join(unhealthy_raw[:min(len(unhealthy_raw), 10)])

In [28]:
healthy_raw = "SM337197,GSM337198,GSM337199,GSM337200,GSM337201,GSM337202,GSM337203,GSM337204,GSM337205,GSM337206,GSM337207,GSM337208,GSM337209,GSM337210,GSM337211,GSM337212,GSM337213,GSM337214,GSM337215,GSM337216,GSM337217,GSM337218,GSM337219,GSM337220,GSM337221,GSM337222,GSM337223,GSM337224,GSM337225,GSM337226,GSM337227,GSM337228,GSM337229,GSM337230,GSM337231,GSM337232,GSM337233,GSM337234,GSM337235,GSM337236,GSM337237,GSM337238,GSM337239,GSM337240,GSM337241,GSM337242,GSM337243,GSM337244,GSM337245,GSM337246,GSM337247,GSM337248,GSM337249,GSM337250,GSM337251,GSM337252,GSM337253,GSM337254,GSM337255,GSM337256,GSM337257,GSM337258,GSM337259,GSM337260"\
            .split(",")
healthy_raw = "\n".join(healthy_raw[:min(len(healthy_raw), 10)])

In [29]:
X, y = parse_database(psoriasis_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("psoriasis_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done   6 out of  19 | elapsed: 17.5min remaining: 37.9min
[Parallel(n_jobs=-1)]: Done   8 out of  19 | elapsed: 17.7min remaining: 24.3min
[Parallel(n_jobs=-1)]: Done  10 out of  19 | elapsed: 19.1min remaining: 17.2min
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed: 19.1min remaining: 11.2min
[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed: 19.2min remaining:  6.8min
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed: 19.3min remaining:  3.6min
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed: 19.9min finished


GSM337198 added with length 19266
GSM337200 added with length 19266
GSM337262 added with length 19266
GSM337269 added with length 19266
GSM337202 added with length 19266
GSM337264 added with length 19266
GSM337268 added with length 19266
GSM337261 added with length 19266
GSM337205 added with length 19266
GSM337206 added with length 19266
GSM337203 added with length 19266
GSM337270 added with length 19266
GSM337266 added with length 19266
GSM337199 added with length 19266
GSM337263 added with length 19266
GSM337265 added with length 19266
GSM337204 added with length 19266
GSM337267 added with length 19266
GSM337201 added with length 19266


## Leukemia

In [32]:
leukemia_dataset = GEOparse.get_GEO(filepath='../datasets/GDS3057_full.soft').table

05-Feb-2018 23:40:48 INFO GEOparse - Parsing ../datasets/GDS3057_full.soft: 
05-Feb-2018 23:40:48 DEBUG GEOparse - DATABASE: Geo
05-Feb-2018 23:40:48 DEBUG GEOparse - DATASET: GDS3057
05-Feb-2018 23:40:48 DEBUG GEOparse - SUBSET: GDS3057_1
05-Feb-2018 23:40:48 DEBUG GEOparse - SUBSET: GDS3057_2
05-Feb-2018 23:40:48 DEBUG GEOparse - SUBSET: GDS3057_3
05-Feb-2018 23:40:48 DEBUG GEOparse - SUBSET: GDS3057_4
05-Feb-2018 23:40:48 DEBUG GEOparse - SUBSET: GDS3057_5
05-Feb-2018 23:40:48 DEBUG GEOparse - SUBSET: GDS3057_6
05-Feb-2018 23:40:48 DEBUG GEOparse - ANNOTATION: 
05-Feb-2018 23:40:48 ERROR GEOparse - Cannot recognize type Annotation
05-Feb-2018 23:40:48 DEBUG GEOparse - DATASET: GDS3057
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/usr/local/lib/python3.5/di

In [30]:
healthy_raw = "GSM239323,GSM239324,GSM239326,GSM239328,GSM239329,GSM239331,GSM239332,GSM239333,GSM239334,GSM239335,GSM239170,GSM239338,GSM239339,GSM239340,GSM239341,GSM239342,GSM239343,GSM239344,GSM240500,GSM240501,GSM240502,GSM240503,GSM240504,GSM240505,GSM240506,GSM240507,GSM240508,GSM240509,GSM240430,GSM240431,GSM240432,GSM240433,GSM240494,GSM240495,GSM240496,GSM240497,GSM240498,GSM240499"\
.split(",")
healthy_raw = "\n".join(healthy_raw[:min(len(healthy_raw), 10)])

In [31]:
unhealthy_raw = "GSM239516,GSM239371,GSM239487,GSM239489,GSM239492,GSM239497,GSM239520,GSM240427,GSM239345,GSM239346,GSM239348,GSM239363,GSM239460,GSM239485,GSM239488,GSM239490,GSM239491,GSM239493,GSM239494,GSM239495,GSM239496,GSM239498,GSM239580,GSM240405,GSM240406,GSM240429"\
.split(",")
unhealthy_raw = "\n".join(unhealthy_raw[:min(len(unhealthy_raw), 10)])

In [33]:
X, y = parse_database(leukemia_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("leukemia_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  7.5min remaining: 11.2min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  7.7min remaining:  6.3min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:  7.8min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  7.9min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  7.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  7.9min finished


GSM239497 added with length 12666
GSM239329 added with length 12666
GSM239334 added with length 12666
GSM239346 added with length 12666
GSM239487 added with length 12666
GSM239516 added with length 12666
GSM239345 added with length 12666
GSM239492 added with length 12666
GSM239328 added with length 12666
GSM240427 added with length 12666
GSM239371 added with length 12666
GSM239489 added with length 12666
GSM239331 added with length 12666
GSM239520 added with length 12666
GSM239324 added with length 12666
GSM239323 added with length 12666
GSM239333 added with length 12666
GSM239335 added with length 12666
GSM239326 added with length 12666
GSM239332 added with length 12666


## Autism
![title](aut.jpg)

In [34]:
autism_dataset = GEOparse.get_GEO(filepath='../datasets/GDS4431_full.soft').table

05-Feb-2018 23:48:43 INFO GEOparse - Parsing ../datasets/GDS4431_full.soft: 
05-Feb-2018 23:48:43 DEBUG GEOparse - DATABASE: Geo
05-Feb-2018 23:48:43 DEBUG GEOparse - DATASET: GDS4431
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_1
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_2
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_3
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_4
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_5
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_6
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_7
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_8
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_9
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_10
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_11
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_12
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_13
05-Feb-2018 23:48:43 DEBUG GEOparse - SUBSET: GDS4431_14
05-Feb-2018 23:48:43 DEBUG 

In [35]:
healthy_raw = "GSM627071,GSM627072,GSM627073,GSM627074,GSM627075,GSM627076,GSM627077,GSM627078,GSM627079,GSM627080,GSM627081,GSM627082,GSM627083,GSM627084,GSM627085,GSM627086,GSM627087,GSM627088,GSM627089,GSM627090,GSM627091,GSM627092,GSM627093,GSM627094,GSM627095,GSM627096,GSM627097,GSM627098,GSM627099,GSM627100,GSM627101,GSM627102,GSM627103,GSM627104,GSM627105,GSM627106,GSM627107,GSM627108,GSM627109,GSM627110,GSM627111,GSM627112,GSM627113,GSM627114,GSM627115,GSM627116,GSM627117,GSM627118,GSM627119,GSM627120,GSM627121,GSM627122,GSM627123,GSM627124,GSM627125,GSM627126,GSM627127,GSM627128,GSM627129,GSM627130,GSM627131,GSM627132,GSM627133,GSM627134,GSM627136,GSM627137,GSM627148,GSM627174,GSM627177"\
.split(",")
healthy_raw = "\n".join(healthy_raw[:min(len(healthy_raw), 10)])

In [36]:
unhealthy_raw = "GSM627135,GSM627138,GSM627139,GSM627140,GSM627141,GSM627142,GSM627143,GSM627144,GSM627145,GSM627146,GSM627147,GSM627149,GSM627150,GSM627151,GSM627152,GSM627153,GSM627154,GSM627155,GSM627156,GSM627157,GSM627158,GSM627159,GSM627160,GSM627161,GSM627162,GSM627163,GSM627164,GSM627165,GSM627166,GSM627167,GSM627168,GSM627169,GSM627170,GSM627171,GSM627172,GSM627173,GSM627175,GSM627176,GSM627178,GSM627179,GSM627180,GSM627181,GSM627182,GSM627183,GSM627184,GSM627185,GSM627186,GSM627187,GSM627188,GSM627189,GSM627190,GSM627191,GSM627192,GSM627193,GSM627194,GSM627195,GSM627196,GSM627197,GSM627198,GSM627199,GSM627200,GSM627201,GSM627202,GSM627203,GSM627204,GSM627205,GSM627206,GSM627207,GSM627208,GSM627209,GSM627210,GSM627211,GSM627212,GSM627213,GSM627214,GSM627215,GSM627216"\
.split(",")
unhealthy_raw = "\n".join(unhealthy_raw[:min(len(unhealthy_raw), 10)])

In [37]:
X, y = parse_database(autism_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("autism_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed: 16.6min remaining: 25.0min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed: 18.0min remaining: 14.8min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed: 18.2min remaining:  7.8min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 19.1min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.1min finished


GSM627080 added with length 19266
GSM627073 added with length 19266
GSM627144 added with length 19266
GSM627078 added with length 19266
GSM627075 added with length 19266
GSM627077 added with length 19266
GSM627142 added with length 19266
GSM627138 added with length 19266
GSM627145 added with length 19266
GSM627074 added with length 19266
GSM627079 added with length 19266
GSM627135 added with length 19266
GSM627140 added with length 19266
GSM627141 added with length 19266
GSM627072 added with length 19266
GSM627143 added with length 19266
GSM627146 added with length 19266
GSM627071 added with length 19266
GSM627139 added with length 19266
GSM627076 added with length 19266


## Endometriosis

In [38]:
healthy_raw = "GSM572761,GSM572752,GSM572753,GSM572754,GSM572755,GSM572756,GSM572762,GSM572764,GSM572757"\
.split(",")
healthy_raw = "\n".join(healthy_raw[:min(len(healthy_raw), 10)])

In [39]:
unhealthy_raw = "GSM572765,GSM572747,GSM572748,GSM572749,GSM572750,GSM572751,GSM572763,GSM572758,GSM572759,GSM572760"\
.split(",")
unhealthy_raw = "\n".join(unhealthy_raw[:min(len(unhealthy_raw), 10)])

In [43]:
endometriosis_dataset = GEOparse.get_GEO(filepath='../datasets/GDS3975_full.soft').table

06-Feb-2018 00:40:07 INFO GEOparse - Parsing ../datasets/GDS3975_full.soft: 
06-Feb-2018 00:40:07 DEBUG GEOparse - DATABASE: Geo
06-Feb-2018 00:40:07 DEBUG GEOparse - DATASET: GDS3975
06-Feb-2018 00:40:07 DEBUG GEOparse - SUBSET: GDS3975_1
06-Feb-2018 00:40:07 DEBUG GEOparse - SUBSET: GDS3975_2
06-Feb-2018 00:40:07 DEBUG GEOparse - ANNOTATION: 
06-Feb-2018 00:40:07 ERROR GEOparse - Cannot recognize type Annotation
06-Feb-2018 00:40:07 DEBUG GEOparse - DATASET: GDS3975
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [41]:
X, y = parse_database(endometriosis_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("endometriosis_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done   6 out of  19 | elapsed: 22.2min remaining: 48.2min
[Parallel(n_jobs=-1)]: Done   8 out of  19 | elapsed: 22.4min remaining: 30.9min
[Parallel(n_jobs=-1)]: Done  10 out of  19 | elapsed: 23.2min remaining: 20.9min
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed: 23.5min remaining: 13.7min
[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed: 23.5min remaining:  8.4min
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed: 23.6min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed: 24.2min finished


GSM572755 added with length 18375
GSM572750 added with length 18375
GSM572759 added with length 18375
GSM572747 added with length 18375
GSM572749 added with length 18375
GSM572748 added with length 18375
GSM572762 added with length 18375
GSM572765 added with length 18375
GSM572753 added with length 18375
GSM572758 added with length 18375
GSM572751 added with length 18375
GSM572761 added with length 18375
GSM572752 added with length 18375
GSM572757 added with length 18375
GSM572754 added with length 18375
GSM572764 added with length 18375
GSM572760 added with length 18375
GSM572763 added with length 18375
GSM572756 added with length 18375


## Oral Carcinoma

In [51]:
oral_carcinoma_dataset = GEOparse.get_GEO(filepath='../datasets/GDS1584_full.soft').table

06-Feb-2018 00:53:36 INFO GEOparse - Parsing ../datasets/GDS1584_full.soft: 
06-Feb-2018 00:53:36 DEBUG GEOparse - DATABASE: Geo
06-Feb-2018 00:53:36 DEBUG GEOparse - DATASET: GDS1584
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_1
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_2
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_3
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_4
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_5
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_6
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_7
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_8
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_9
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_10
06-Feb-2018 00:53:36 DEBUG GEOparse - SUBSET: GDS1584_11
06-Feb-2018 00:53:36 DEBUG GEOparse - ANNOTATION: 
06-Feb-2018 00:53:36 ERROR GEOparse - Cannot recognize type Annotation
06-Feb-2018 00:53:36 DEBUG GEOparse - DATASET: GDS1584
/usr/local/lib/python

In [46]:
healthy_raw = "GSM80476,GSM80477,GSM80520,GSM80521"\
.split(",")
healthy_raw = "\n".join(healthy_raw[:min(len(healthy_raw), 10)])

In [45]:
unhealthy_raw = "GSM80460,GSM80461,GSM80462,GSM80463,GSM80464,GSM80465,GSM80466,GSM80467,GSM80468,GSM80469,GSM80470,GSM80471,GSM80472,GSM80473,GSM80474,GSM80475"\
.replace(",","\n")

In [48]:
X, y = parse_database(oral_carcinoma_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("oral_carcinoma_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  5.1min remaining:  7.6min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  5.1min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:  5.2min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  5.2min remaining:   55.3s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  5.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  5.2min finished


GSM80465 added with length 9245
GSM80460 added with length 9245
GSM80474 added with length 9245
GSM80466 added with length 9245
GSM80468 added with length 9245
GSM80461 added with length 9245
GSM80467 added with length 9245
GSM80472 added with length 9245
GSM80477 added with length 9245
GSM80476 added with length 9245
GSM80475 added with length 9245
GSM80469 added with length 9245
GSM80520 added with length 9245
GSM80521 added with length 9245
GSM80473 added with length 9245
GSM80470 added with length 9245
GSM80463 added with length 9245
GSM80462 added with length 9245
GSM80464 added with length 9245
GSM80471 added with length 9245


## Colorectal Cancer

In [50]:
colorectal_cancer_dataset = GEOparse.get_GEO(filepath='../datasets/GDS2918_full.soft').table

06-Feb-2018 00:53:30 INFO GEOparse - Parsing ../datasets/GDS2918_full.soft: 
06-Feb-2018 00:53:30 DEBUG GEOparse - DATABASE: Geo
06-Feb-2018 00:53:30 DEBUG GEOparse - DATASET: GDS2918
06-Feb-2018 00:53:30 DEBUG GEOparse - SUBSET: GDS2918_1
06-Feb-2018 00:53:30 DEBUG GEOparse - SUBSET: GDS2918_2
06-Feb-2018 00:53:30 DEBUG GEOparse - ANNOTATION: 
06-Feb-2018 00:53:30 ERROR GEOparse - Cannot recognize type Annotation
06-Feb-2018 00:53:30 DEBUG GEOparse - DATASET: GDS2918
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [53]:
healthy_raw = "GSM112309,GSM112310,GSM112311,GSM112312,GSM112313,GSM112314,GSM112315,GSM112316"\
.split(",")
healthy_raw = "\n".join(healthy_raw[:min(len(healthy_raw), 10)])

In [54]:
unhealthy_raw = "GSM112207,GSM112208,GSM112299,GSM112300,GSM112301,GSM112302,GSM112303,GSM112304,GSM112305,GSM112306,GSM112307,GSM112308"\
.replace(",","\n")

In [55]:
X, y = parse_database(colorectal_cancer_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("colorectal_cancer_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  6.5min remaining:  9.8min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  6.5min remaining:  5.3min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:  6.6min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  6.6min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  6.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  6.6min finished


GSM112303 added with length 4805
GSM112313 added with length 4805
GSM112316 added with length 4805
GSM112302 added with length 4805
GSM112304 added with length 4805
GSM112309 added with length 4805
GSM112208 added with length 4805
GSM112299 added with length 4805
GSM112312 added with length 4805
GSM112306 added with length 4805
GSM112307 added with length 4805
GSM112315 added with length 4805
GSM112207 added with length 4805
GSM112300 added with length 4805
GSM112314 added with length 4805
GSM112310 added with length 4805
GSM112308 added with length 4805
GSM112305 added with length 4805
GSM112311 added with length 4805
GSM112301 added with length 4805


## Cervical Cancer

In [56]:
cervical_cancer_dataset = GEOparse.get_GEO(filepath='../datasets/GDS470_full.soft').table

06-Feb-2018 01:02:40 INFO GEOparse - Parsing ../datasets/GDS470_full.soft: 
06-Feb-2018 01:02:40 DEBUG GEOparse - DATABASE: Geo
06-Feb-2018 01:02:40 DEBUG GEOparse - DATASET: GDS470
06-Feb-2018 01:02:40 DEBUG GEOparse - SUBSET: GDS470_1
06-Feb-2018 01:02:40 DEBUG GEOparse - SUBSET: GDS470_2
06-Feb-2018 01:02:40 DEBUG GEOparse - SUBSET: GDS470_3
06-Feb-2018 01:02:40 DEBUG GEOparse - SUBSET: GDS470_4
06-Feb-2018 01:02:40 DEBUG GEOparse - SUBSET: GDS470_5
06-Feb-2018 01:02:40 DEBUG GEOparse - ANNOTATION: 
06-Feb-2018 01:02:40 ERROR GEOparse - Cannot recognize type Annotation
06-Feb-2018 01:02:40 DEBUG GEOparse - DATASET: GDS470
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [57]:
healthy_raw = """
#GSM7841 = Value for GSM7841: Normal Cervix N10; src: Normal Cervix
#GSM7842 = Value for GSM7842: Normal Cervix N13; src: Normal Cervix
#GSM7843 = Value for GSM7843: Normal Cervix N14; src: Normal Cervix
#GSM7844 = Value for GSM7844: Normal Cervix N17; src: Normal Cervix
#GSM7845 = Value for GSM7845: Normal Cervix N5; src: Normal Cervix
#GSM7846 = Value for GSM7846: Normal Cervix N6; src: Normal Cervix
#GSM7847 = Value for GSM7847: Normal Cervix N8; src: Normal Cervix
#GSM7848 = Value for GSM7848: Normal Cervix N9; src: Normal Cervix
"""

In [59]:
unhealthy_raw = """
#GSM7828 = Value for GSM7828: Cervical Cancer C482; src: Cervical cancer tissue
#GSM7830 = Value for GSM7830: Cervical Cancer C494; src: Cervical cancer tissue
#GSM7834 = Value for GSM7834: Cervical Cancer C502; src: Cervical cancer tissue
#GSM7836 = Value for GSM7836: Cervical Cancer C517; src: Cervical cancer tissue
#GSM7837 = Value for GSM7837: Cervical Cancer C524; src: Cervical cancer tissue
#GSM7838 = Value for GSM7838: Cervical Cancer C525; src: Cervical cancer tissue
#GSM7840 = Value for GSM7840: Cervical Cancer C540; src: Cervical cancer tissue
#GSM7854 = Value for GSM7854: Cervical Cancer C542; src: Cervical cancer tissue
#GSM7855 = Value for GSM7855: Cervical Cancer C555; src: Cervical cancer tissue
#GSM7856 = Value for GSM7856: Cervical Cancer C564; src: Cervical cancer tissue
#GSM7824 = Value for GSM7824: Cervical Cancer C437; src: Cervical cancer tissue
#GSM7827 = Value for GSM7827: Cervical Cancer C477; src: Cervical cancer tissue
"""

In [61]:
X, y = parse_database(cervical_cancer_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("cervical_cancer_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  1.4min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  1.4min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:  1.4min remaining:   36.2s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  1.4min remaining:   14.9s


GSM7844 added with length 1227
GSM7830 added with length 1227
GSM7845 added with length 1227
GSM7854 added with length 1227
GSM7846 added with length 1227
GSM7838 added with length 1227
GSM7824 added with length 1227
GSM7847 added with length 1227
GSM7848 added with length 1227
GSM7836 added with length 1227
GSM7827 added with length 1227
GSM7828 added with length 1227
GSM7837 added with length 1227
GSM7856 added with length 1227
GSM7840 added with length 1227
GSM7834 added with length 1227
GSM7841 added with length 1227
GSM7842 added with length 1227
GSM7855 added with length 1227
GSM7843 added with length 1227


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.4min finished


## Brain Tumours

In [62]:
brain_dataset = GEOparse.get_GEO(filepath='../datasets/GDS4467_full.soft').table

06-Feb-2018 01:32:56 INFO GEOparse - Parsing ../datasets/GDS4467_full.soft: 
06-Feb-2018 01:32:56 DEBUG GEOparse - DATABASE: Geo
06-Feb-2018 01:32:56 DEBUG GEOparse - DATASET: GDS4467
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_1
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_2
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_3
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_4
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_5
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_6
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_7
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_8
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_9
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_10
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_11
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_12
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_13
06-Feb-2018 01:32:56 DEBUG GEOparse - SUBSET: GDS4467_14
06-Feb-2018 01:32:56 DEBUG 

In [66]:
unhealthy_raw = """
#GSM397648 = Value for GSM397648: Oligodendrioglioma_3; src: brain tumor tissue
#GSM397649 = Value for GSM397649: Oligodendrioglioma_4; src: brain tumor tissue
#GSM397652 = Value for GSM397652: Oligodendrioglioma_7; src: brain tumor tissue
#GSM397639 = Value for GSM397639: Astrocytoma_2; src: brain tumor tissue
#GSM397640 = Value for GSM397640: Astrocytoma_3; src: brain tumor tissue
#GSM397642 = Value for GSM397642: Astrocytoma_5; src: brain tumor tissue
#GSM397645 = Value for GSM397645: Astrocytoma_8; src: brain tumor tissue
#GSM397633 = Value for GSM397633: Glioblastoma_8; src: brain tumor tissue
#GSM397634 = Value for GSM397634: Glioblastoma_9; src: brain tumor tissue
#GSM397635 = Value for GSM397635: Glioblastoma_10; src: brain tumor tissue
#GSM397636 = Value for GSM397636: Glioblastoma_11; src: brain tumor tissue
#GSM397637 = Value for GSM397637: Glioblastoma_12; src: brain tumor tissue
#GSM397653 = Value for GSM397653: secondary glioblastoma_1; src: brain tumor tissue
#GSM397654 = Value for GSM397654: secondary glioblastoma_2; src: brain tumor tissue
#GSM397655 = Value for GSM397655: secondary glioblastoma_3; src: brain tumor tissue
"""

In [67]:
healthy_raw = """
#GSM397656 = Value for GSM397656: normal astrocytes_1; src: normal astrocytes
#GSM397657 = Value for GSM397657: normal astrocytes_2; src: normal astrocytes
#GSM397658 = Value for GSM397658: normal astrocytes_3; src: normal astrocytes
#GSM397659 = Value for GSM397659: normal brain_1; src: normal brain tissue
#GSM397660 = Value for GSM397660: normal brain_2; src: normal brain tissue"""

In [68]:
X, y = parse_database(brain_dataset, healthy_raw, unhealthy_raw, n_jobs=-1)
pickle.dump({"X":X,"y":y}, open("brain_cancer_Xy", "wb"))

../classes/geno_utils.py:49 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed: 16.9min remaining: 25.3min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed: 18.2min remaining: 14.9min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed: 18.4min remaining:  7.9min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 19.3min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.3min finished


GSM397645 added with length 19266
GSM397652 added with length 19266
GSM397655 added with length 19266
GSM397639 added with length 19266
GSM397660 added with length 19266
GSM397656 added with length 19266
GSM397654 added with length 19266
GSM397649 added with length 19266
GSM397634 added with length 19266
GSM397657 added with length 19266
GSM397636 added with length 19266
GSM397659 added with length 19266
GSM397635 added with length 19266
GSM397640 added with length 19266
GSM397633 added with length 19266
GSM397653 added with length 19266
GSM397642 added with length 19266
GSM397648 added with length 19266
GSM397658 added with length 19266
GSM397637 added with length 19266
